In [1]:
test_mode = False

In [2]:
import pandas as pd
import hopsworks
from utils import *
import json
import os
import warnings
from dotenv import load_dotenv
import datetime
#from datetime import timezone

warnings.filterwarnings("ignore")

In [3]:
load_dotenv()
#os.environ["HOPSWORKS_API_KEY"] = os.getenv("HOPSWORKS_API_KEY")


project = hopsworks.login()

2025-01-02 16:15:33,300 INFO: Initializing external client
2025-01-02 16:15:33,300 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-02 16:15:34,912 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1160346


In [4]:

fs = project.get_feature_store() 
#secrets = secrets_api(project.name)

# This line will fail if you have not registered the CRYPTO_API_KEY as a secret in Hopsworks
#CRYPTO_API_KEY = secrets.get_secret("CRYPTO_API_KEY").value

#CRYPTO_API_KEY = os.getenv("CRYPTO_API_KEY")
today = int(time.time())
seconds_in_one_day = 24 * 60 * 60


today


1735827336

In [5]:

# Step 2: Retrieve the feature group by name and version
solana_feature_group = fs.get_feature_group(
    name="solana",  
    version=1
)

# Step 3: Read the data from the feature group
s_data_df = solana_feature_group.select(["timestamp"]).read()

# Step 4: Find the maximum date
solana_max_date = s_data_df['timestamp'].max()
solana_difference = (today - solana_max_date)//seconds_in_one_day

# Step 5: Print the result
print(f"The maximum date is: {solana_max_date}")
print(f"Difference in days: {solana_difference}")


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.57s) 
The maximum date is: 1734912000
Difference in days: 10


In [6]:
# Step 2: Retrieve the feature group by name and version
bitcoin_feature_group = fs.get_feature_group(
    name="bitcoin",  
    version=1
)

# Step 3: Read the data from the feature group
b_data_df = bitcoin_feature_group.select(["timestamp"]).read()

# Step 4: Find the maximum date
bitcoin_max_date = b_data_df['timestamp'].max()
bitcoin_difference = (today - bitcoin_max_date)//seconds_in_one_day

# Step 5: Print the result
print(f"The maximum date is: {bitcoin_max_date}")
print(f"Difference in days: {bitcoin_difference}")


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.56s) 
The maximum date is: 1734912000
Difference in days: 10


In [7]:
#fs = project.get_feature_store(name='kth_lab1_2024_featurestore')
fs = project.get_feature_store()

fng_feature_group = fs.get_feature_group(
    name='f_n_g_index', 
    version=1)
fng_df = fng_feature_group.read()

# Step 4: Find the maximum date
fng_max_date = int(fng_df['timestamp'].max())
fng_difference = (today - fng_max_date)//seconds_in_one_day

# Step 5: Print the result
print(f"The maximum date is: {fng_max_date}")
print(f"Difference in days: {fng_difference}")


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.87s) 
The maximum date is: 1735776000
Difference in days: 0


### Getting missing data

### Request solana price from last available data till now

In [8]:

url = "https://data-api.cryptocompare.com//spot/v1/historical/days"

# Load environment variables from the .env file
load_dotenv()
crypto_api_key = os.getenv("CRYPTO_API_KEY")


params = {
    "market":"kraken",
    "instrument": "SOL-USD",  
    "limit": solana_difference,  
    "api_key": crypto_api_key,
}
if solana_difference!=0:
    sol_response = trigger_request(url, params)
    sol_response_df = pd.DataFrame(sol_response["Data"])
    sol_response_df = sol_response_df[["TIMESTAMP", 'OPEN', 'HIGH', 'LOW', 'CLOSE', "VOLUME", 'VOLUME_BUY', 'VOLUME_SELL']]
    sol_response_df.columns = sol_response_df.columns.str.lower()
    print("Num entries to add: ", len(sol_response_df))
else:
    sol_response_df = pd.DataFrame(columns=["timestamp","open","high", "low","close", "volume", "volume_buy", "volume_sell"])

Num entries to add:  10


In [9]:
sol_response_df.head()

,timestamp,open,high,low,close,volume,volume_buy,volume_sell
0,1734998400,189.91,198.70,185.97,196.92,147273.311796,72402.790065,74870.521731
1,1735084800,196.92,201.65,195.08,197.38,77888.393938,39038.188456,38850.205482
2,1735171200,197.38,199.54,186.50,188.27,126419.530813,48442.972388,77976.558425
3,1735257600,188.27,194.00,182.62,183.81,134928.650993,69516.047164,65412.603829
4,1735344000,183.81,195.58,183.57,195.15,107692.510497,67914.329251,39778.181246


### Request bitcoin price from last available data till now

In [10]:
load_dotenv()
crypto_api_key = os.getenv("CRYPTO_API_KEY")


params = {
    "market":"kraken",
    "instrument": "BTC-USD",  
    "limit": bitcoin_difference,  
    "api_key": crypto_api_key,
}

if bitcoin_difference>0:
    bitcoin_response = trigger_request(url, params)
    bitcoin_response_df = pd.DataFrame(bitcoin_response["Data"])
    bitcoin_response_df = bitcoin_response_df[["TIMESTAMP", 'OPEN', 'HIGH', 'LOW', 'CLOSE', "VOLUME", 'VOLUME_BUY', 'VOLUME_SELL']]
    bitcoin_response_df.columns = bitcoin_response_df.columns.str.lower()
    print("Num entries to add: ", len(bitcoin_response_df))
else:
    bitcoin_response_df = pd.DataFrame(columns=["timestamp","open","high", "low","close", "volume", "volume_buy", "volume_sell"])

bitcoin_response_df.head()

Num entries to add:  10


,timestamp,open,high,low,close,volume,volume_buy,volume_sell
0,1734998400,94699.4,99400.0,93500.0,98647.2,1533.799822,891.166136,642.633687
1,1735084800,98647.2,99411.6,97600.0,99310.8,681.610471,410.423060,271.187412
2,1735171200,99310.8,99821.8,95139.1,95645.0,1286.630454,525.643653,760.986801
3,1735257600,95645.0,97308.6,93300.0,94148.9,1781.178175,1005.508540,775.669635
4,1735344000,94148.9,95497.0,94033.5,95204.1,447.871951,237.945173,209.926778


In [11]:
bitcoin_response_df

,timestamp,open,high,low,close,volume,volume_buy,volume_sell
0,1734998400,94699.4,99400.0,93500.0,98647.2,1533.799822,891.166136,642.633687
1,1735084800,98647.2,99411.6,97600.0,99310.8,681.610471,410.423060,271.187412
2,1735171200,99310.8,99821.8,95139.1,95645.0,1286.630454,525.643653,760.986801
3,1735257600,95645.0,97308.6,93300.0,94148.9,1781.178175,1005.508540,775.669635
4,1735344000,94148.9,95497.0,94033.5,95204.1,447.871951,237.945173,209.926778
5,1735430400,95204.1,95204.1,92860.7,93551.0,698.547127,229.720452,468.826675
6,1735516800,93551.0,94858.7,91256.6,92611.3,2168.291728,994.214537,1174.077191
7,1735603200,92611.3,96040.0,91934.1,93370.9,1697.950414,943.514778,754.435636
8,1735689600,93370.9,94888.0,92588.1,94380.8,576.162219,306.250085,269.912134
9,1735776000,94380.8,96767.3,94167.2,96175.1,755.579919,463.460716,292.119203


### Request fear and greed index values from last available data till now

In [12]:
url = "https://api.alternative.me/fng/?"

# Set the parameters for the API request
params = {
    "limit": fng_difference  # Number of days of data to retrieve
}

if fng_difference>0:
    fng_response = trigger_request(url, params)
    fng_df = pd.DataFrame(fng_response['data'])
    fng_df['timestamp'] = fng_df['timestamp'].astype(float)
    fng_df.rename(columns={'value_classification': 'fng_classification'}, inplace=True)
    fng_df.rename(columns={'value': 'fng_value'}, inplace=True)

    fng_df.drop(columns=["time_until_update"],inplace=True)
    fng_df.head()
else:
    fng_df = pd.DataFrame(columns=["fng_value","fng_classification","timestamp"])
fng_df.head()

,fng_value,fng_classification,timestamp


### Now we need to use the responses to update the data in Hopsworks

In [13]:
if not test_mode and sol_response_df.shape[0]>0:
    solana_feature_group.insert(sol_response_df)



Uploading Dataframe: 100.00% |██████████| Rows 10/10 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: solana_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1160346/jobs/named/solana_1_offline_fg_materialization/executions


In [14]:
if not test_mode and bitcoin_response_df.shape[0]>0:
    bitcoin_feature_group.insert(bitcoin_response_df)


Uploading Dataframe: 100.00% |██████████| Rows 10/10 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: bitcoin_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1160346/jobs/named/bitcoin_1_offline_fg_materialization/executions


In [15]:
fng_df.head()

,fng_value,fng_classification,timestamp


In [16]:
if not test_mode and fng_df.shape[0]>0:
    fng_feature_group.insert(fng_df)